In [53]:
# from langchain_community.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader
from langchain_community.document_loaders import PyPDFLoader
from pprint import pprint
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma

In [54]:
local_path = "/mnt/c/Users/user/OneDrive/Desktop/rag-chatbot-telegram-data/data/WEF_The_Global_Cooperation_Barometer_2024.pdf"

In [64]:
if local_path:
    loader = PyPDFLoader(file_path=local_path)
    data = loader.load()
else:
    print("Upload a PDF file")
    
text_splitter = RecursiveCharacterTextSplitter(chunk_size=7500, chunk_overlap=100)
chunks = text_splitter.split_documents(data)

vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text", show_progress=True),
    collection_name='local-rag')
    

OllamaEmbeddings:   0%|          | 0/26 [00:00<?, ?it/s]

ValueError: Error raised by inference API HTTP code: 500, {"error":"llama runner: failed to load model '/usr/share/ollama/.ollama/models/blobs/sha256:970aa74c0a90ef7482477cf803618e776e173c007bf957f635f1015bfcfef0e6': this model may be incompatible with your version of Ollama. If you previously pulled this model, try updating it by running `ollama pull nomic-embed-text:latest`"}

In [52]:
# pprint(data[4].page_content)